# Deploying functions 

In [118]:
#Reference
#https://docs.openfaas.com/tutorials/kubernetes-hpa/
#!ssh ubuntu@172.17.141.197 "sudo kubectl port-forward -n openfaas svc/gateway 8080:8080 &"
import warnings

warnings.filterwarnings('ignore')

In [119]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f aeneas.yml  --env max_inflight=50 --annotation topic="aeneas" --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1"

Deploying: aeneas.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/aeneas



In [120]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f scaling-aeneas-tocloud.yml --env max_inflight=50 --annotation topic="tocloud" --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1;faas deploy  -f getobject.yml --env max_inflight=50 --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1;faas deploy  -f pubrabbitmq.yml --env max_inflight=50 --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1"


Deploying: scaling-aeneas-tocloud.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/scaling-aeneas-tocloud

Deploying: getobject.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/getobject

Deploying: pubrabbitmq.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/pubrabbitmq



In [121]:
!ssh ubuntu@172.17.141.197 "faas describe getobject"

Name:               getobject
Status:             Ready
Replicas:           1
Available Replicas: 1
Invocations:        12702
Image:              shivupoojar/scalinggetobject:latest
Function Process:   python index.py
URL:                http://127.0.0.1:8080/function/getobject
Async URL:          http://127.0.0.1:8080/async-function/getobject
Labels:
 com.openfaas.scale.max: 1
 com.openfaas.scale.min: 1
 faas_function: getobject
 uid: 568007617
Annotations:
 prometheus.io.scrape: false
Requests:  CPU: 500m
 Memory: 128Mi


# Deploying Rabbitmq Connector 

In [122]:
# Deploying connector configurartion
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply -f cf.yaml"

configmap/rabbitmq-connector-configmap unchanged


In [123]:
# Deploy connector
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply -f configmap.yaml"

deployment.apps/rabbitmq-connector configured


In [124]:
!ssh ubuntu@172.17.141.197 "sudo kubectl get po -n openfaas" 

NAME                                  READY   STATUS    RESTARTS         AGE
nats-5b77986b8-48p2x                  1/1     Running   4 (5d21h ago)    42d
queue-worker-7d8668444f-tmq7b         1/1     Running   5 (5d21h ago)    27d
alertmanager-6cd7bd5c64-hrcrt         1/1     Running   4 (5d21h ago)    27d
basic-auth-plugin-78b958969c-tnpck    1/1     Running   4 (5d21h ago)    27d
gateway-bf4f5d6f7-4q7kn               2/2     Running   10 (4d16h ago)   27d
prometheus-7678c644d6-k28wr           1/1     Running   0                36m
rabbitmq-connector-64cbd548f5-vtw26   1/1     Running   0                33m


# Deploying k8s HPA components 

In [125]:
!ssh ubuntu@172.17.141.197 "sudo kubectl autoscale deployment -n openfaas-fn aeneas --cpu-percent=50 --min=1 --max=15;sudo kubectl autoscale deployment -n openfaas-fn scaling-aeneas-tocloud --cpu-percent=50 --min=1 --max=10;sudo kubectl autoscale deployment -n openfaas rabbitmq-connector --cpu-percent=50 --min=1 --max=10;sudo kubectl autoscale deployment -n openfaas-fn getobject --cpu-percent=50 --min=1 --max=10;sudo kubectl autoscale deployment -n openfaas-fn pubrabbitmq --cpu-percent=50 --min=1 --max=10"

Error from server (AlreadyExists): horizontalpodautoscalers.autoscaling "aeneas" already exists
Error from server (AlreadyExists): horizontalpodautoscalers.autoscaling "scaling-aeneas-tocloud" already exists
Error from server (AlreadyExists): horizontalpodautoscalers.autoscaling "rabbitmq-connector" already exists
Error from server (AlreadyExists): horizontalpodautoscalers.autoscaling "getobject" already exists
Error from server (AlreadyExists): horizontalpodautoscalers.autoscaling "pubrabbitmq" already exists


In [126]:
#Check for the created HPA
!ssh ubuntu@172.17.141.197 "sudo kubectl get hpa -n openfaas-fn"

NAME                     REFERENCE                           TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
aeneas                   Deployment/aeneas                   0%/50%    1         15        1          13m
scaling-aeneas-tocloud   Deployment/scaling-aeneas-tocloud   0%/50%    1         10        1          13m
getobject                Deployment/getobject                0%/50%    1         10        1          13m
pubrabbitmq              Deployment/pubrabbitmq              0%/50%    1         10        1          13m


# Experiments 

In [127]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

In [128]:
scenario = "k8s"
user=10
iteration=0.05
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:31022/'
time_locust=120


In [129]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    client.remove_object(bucket_name, obj.object_name)

In [130]:
!rm input_data.csv
!touch input_data.csv
!echo "intime" > input_data.csv

In [131]:
!locust --headless -f testingrabbitlocust.py --host='127.0.0.1' --skip-log -t 120s  --users 10 --spawn-rate 0.05

In [132]:
time.sleep(60)

In [133]:
df= pd.read_csv('input_data.csv')
print(df)

                          intime
0   '2023-01-17 11:11:01.107492'
1   '2023-01-17 11:11:07.067946'
2   '2023-01-17 11:11:20.591542'
3   '2023-01-17 11:11:21.099354'
4   '2023-01-17 11:11:30.562479'
5   '2023-01-17 11:11:32.546043'
6   '2023-01-17 11:11:39.287348'
7   '2023-01-17 11:11:41.099720'
8   '2023-01-17 11:11:45.225100'
9   '2023-01-17 11:11:45.339529'
10  '2023-01-17 11:11:47.148837'
11  '2023-01-17 11:11:55.597568'
12  '2023-01-17 11:11:57.942127'
13  '2023-01-17 11:12:00.390908'
14  '2023-01-17 11:12:01.096186'
15  '2023-01-17 11:12:01.432222'
16  '2023-01-17 11:12:05.136081'
17  '2023-01-17 11:12:12.119291'
18  '2023-01-17 11:12:12.369145'
19  '2023-01-17 11:12:12.721937'
20  '2023-01-17 11:12:15.214424'
21  '2023-01-17 11:12:19.443126'
22  '2023-01-17 11:12:21.100427'
23  '2023-01-17 11:12:21.903022'
24  '2023-01-17 11:12:22.372281'
25  '2023-01-17 11:12:29.211622'
26  '2023-01-17 11:12:29.220769'
27  '2023-01-17 11:12:33.523824'
28  '2023-01-17 11:12:33.723824'
29  '2023-

In [134]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)#
    client.remove_object(bucket_name, obj.object_name)


In [135]:
outtime
df

,intime
0,'2023-01-17 11:11:01.107492'
1,'2023-01-17 11:11:07.067946'
2,'2023-01-17 11:11:20.591542'
3,'2023-01-17 11:11:21.099354'
4,'2023-01-17 11:11:30.562479'
5,'2023-01-17 11:11:32.546043'
6,'2023-01-17 11:11:39.287348'
7,'2023-01-17 11:11:41.099720'
8,'2023-01-17 11:11:45.225100'
9,'2023-01-17 11:11:45.339529'


In [136]:

df['outtime']= outtime
df['outtime']= pd.to_datetime(df['outtime'],errors='coerce').dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'],errors='coerce')
#df =df.sort_values(by="outtime")
display(df)
print(df['outtime'].iloc[-1]-df['intime'].iloc[0])

,intime,outtime
0,2023-01-17 11:11:01.107492,2023-01-17 11:11:05.490
1,2023-01-17 11:11:07.067946,2023-01-17 11:11:11.282
2,2023-01-17 11:11:20.591542,2023-01-17 11:11:27.356
3,2023-01-17 11:11:21.099354,2023-01-17 11:11:27.512
4,2023-01-17 11:11:30.562479,2023-01-17 11:11:35.939
5,2023-01-17 11:11:32.546043,2023-01-17 11:11:38.274
6,2023-01-17 11:11:39.287348,2023-01-17 11:11:44.565
7,2023-01-17 11:11:41.099720,2023-01-17 11:11:47.105
8,2023-01-17 11:11:45.225100,2023-01-17 11:11:56.355
9,2023-01-17 11:11:45.339529,2023-01-17 11:11:56.796


0 days 00:02:26.420508


In [137]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))


In [138]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn  -n openfaas -c gateway  | grep /function/aeneas | grep 200 | cut -c 80-83 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
cmd3="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/getobject | grep POST | cut -c 71-76  | tail -"+str(len(outtime))
cmd4="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/pubrabbitmq | grep POST | cut -c 73-78 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
#df['aeneas']=(pd.to_timedelta(df['aeneas'].astype(float), unit='s'))

df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]
#df['tocloud']=(pd.to_timedelta(df['tocloud'].astype(float), unit='s'))

#df['getobject']=(get_function_execution_time(cmd3)).split("\n")[:-1]
df['pubrabbitmq']=((get_function_execution_time(cmd4)).split("\n")[:-1])
#df['pubrabbitmq']=(pd.to_timedelta(df['pubrabbitmq'].astype(float), unit='s'))
df

,intime,outtime,aeneas,tocloud,pubrabbitmq
0,2023-01-17 11:11:01.107492,2023-01-17 11:11:05.490,4.29,0.0405,0.0353
1,2023-01-17 11:11:07.067946,2023-01-17 11:11:11.282,4.13,0.0341,0.0361
2,2023-01-17 11:11:20.591542,2023-01-17 11:11:27.356,6.79,0.0385,0.0298
3,2023-01-17 11:11:21.099354,2023-01-17 11:11:27.512,6.32,0.0424,0.0286
4,2023-01-17 11:11:30.562479,2023-01-17 11:11:35.939,5.29,0.0415,0.0379
5,2023-01-17 11:11:32.546043,2023-01-17 11:11:38.274,5.63,0.0469,0.0334
6,2023-01-17 11:11:39.287348,2023-01-17 11:11:44.565,5.18,0.0393,0.0352
7,2023-01-17 11:11:41.099720,2023-01-17 11:11:47.105,5.91,0.0420,0.0314
8,2023-01-17 11:11:45.225100,2023-01-17 11:11:56.355,10.9,0.0547,0.0325
9,2023-01-17 11:11:45.339529,2023-01-17 11:11:56.796,11.4,0.0565,0.0495


In [139]:
#df =df.sort_values(by="outtime")
df['TPT']=(df['outtime']-df['intime'])
df['TPT']=df['TPT'].values.astype('datetime64[ns]')
df['TPT']= pd.to_datetime(df['TPT']).dt.strftime('%S.%f')
df['TPT']=df['TPT'].astype(float)
df['PT']= df['TPT'].round(decimals = 0)
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
#df['MQ time']= ((df['TPT']).astype(float) - df['FET'])
df["scenario"]= ""+str(user)+"_"+str(iteration)+"_"+scenario
display(df)
#TPT: Total Processing Time
#FET: Function Execution Time
#CT: Communication Time
#print(df)
# #df =df.sort_values(by="outtime")
# df['TPT']=(df['outtime']-df['intime'])
# #print(df['TPT'])
# #df['aeneas']= pd.to_timedelta(df['aeneas'].astype(float) , unit='s')
# df['FET'] = df['aeneas'].astype(float) + df['tocloud'].astype(float)
# #df['FET'] = pd.to_timedelta(df['aeneas'].astype(float), unit='s') + pd.to_timedelta(df['tocloud'].astype(float) , unit='s')
# df['FET']= pd.to_timedelta(df['FET'], unit='s')
# df['MQ time']= (df['TPT']) - (df['FET'])
# df["scenario"]= ""+str(user)+"_"+str(iteration)+"_"+scenario
# display(df)
df.to_csv(""+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')
print(df.index)

,intime,outtime,aeneas,tocloud,pubrabbitmq,TPT,PT,FET,scenario
0,2023-01-17 11:11:01.107492,2023-01-17 11:11:05.490,4.29,0.0405,0.0353,4.382508,4.0,4.3305,10_0.05_k8s
1,2023-01-17 11:11:07.067946,2023-01-17 11:11:11.282,4.13,0.0341,0.0361,4.214054,4.0,4.1641,10_0.05_k8s
2,2023-01-17 11:11:20.591542,2023-01-17 11:11:27.356,6.79,0.0385,0.0298,6.764458,7.0,6.8285,10_0.05_k8s
3,2023-01-17 11:11:21.099354,2023-01-17 11:11:27.512,6.32,0.0424,0.0286,6.412646,6.0,6.3624,10_0.05_k8s
4,2023-01-17 11:11:30.562479,2023-01-17 11:11:35.939,5.29,0.0415,0.0379,5.376521,5.0,5.3315,10_0.05_k8s
5,2023-01-17 11:11:32.546043,2023-01-17 11:11:38.274,5.63,0.0469,0.0334,5.727957,6.0,5.6769,10_0.05_k8s
6,2023-01-17 11:11:39.287348,2023-01-17 11:11:44.565,5.18,0.0393,0.0352,5.277652,5.0,5.2193,10_0.05_k8s
7,2023-01-17 11:11:41.099720,2023-01-17 11:11:47.105,5.91,0.0420,0.0314,6.005280,6.0,5.9520,10_0.05_k8s
8,2023-01-17 11:11:45.225100,2023-01-17 11:11:56.355,10.9,0.0547,0.0325,11.129900,11.0,10.9547,10_0.05_k8s
9,2023-01-17 11:11:45.339529,2023-01-17 11:11:56.796,11.4,0.0565,0.0495,11.456471,11.0,11.4565,10_0.05_k8s


RangeIndex(start=0, stop=45, step=1)


In [140]:
pre_url = PROMETHEUS + '/api/v1/query_range?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time_interval = (time.mktime(timestamp.timetuple()))
user=df.shape[0]
end=(time.mktime(timestamp.timetuple()))
start=pd.to_datetime(df['intime'].iloc[0], format='%Y-%m-%d %H:%M:%S')
start=(time.mktime(start.timetuple()))
print(start,end)

1673953861.0 1673954007.0


In [141]:
interval = (df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()
interval= round(interval)

In [142]:
interval= round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) if round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) > 60  else 60
print(interval)

146


In [143]:
import copy 
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    #data2=res.get('data').get('result')[0].get('values')
    data_dict={}
    metric_list = []
    # print(data['data']['result']['values'])
    # exit()
    for i in res['data']['result']:
        for j in i['values']:
            data_dict = copy.deepcopy(i['metric'])
            data_dict['time'] = j[0]
            data_dict['value'] = j[1]
            metric_list.append(data_dict)
    df_metric = pd.DataFrame(metric_list)
    return df_metric

In [144]:
user=df.shape[0]
metrics = {'workload_type':""+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration),'workload_size':user,'Workload TPT':(df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean()}

promo_metrics = pd.DataFrame(columns=['cpu_used','cpu_requested'])

In [145]:

expr_namespace_cpu_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['value']

promo_metrics['time']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['time']
expr_namespace_cpu_used = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)['value']

expr_namespace_pod_count = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['pod_count']=getdataprometheus(pre_url+expr_namespace_pod_count)['value']

expr_namespace_memory_used = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_used)['value']

expr_namespace_memory_requested = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_requested)['value']

promo_metrics['time']= pd.to_datetime(promo_metrics['time'],unit='s')
seconds = []
for i in range(promo_metrics.shape[0]): 
    seconds.append(i*10)
promo_metrics['seconds'] = seconds

throughput_qurty = 'sum (rate(gateway_function_invocation_total{code="200"}[10s]))'
promo_metrics['throughput']= getdataprometheus('http://172.17.141.197:31376/api/v1/query_range?query=sum (rate(gateway_function_invocation_total{code="200"}[20s]))&start='+str(start)+'&end='+str(end)+'&step=10s')['value']
promo_metrics.to_csv("keda_results/promo_metrics"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv',index=False)


# #expr_namespace_cpu_requested = 'sum(node_namespace_pod_container:container_cpu_usage_seconds_total:sum_irate{namespace="openfaas-fn"})&time='+str(time_interval)
# expr_namespace_cpu_used='sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)
# expr_namespace_cpu_requested='sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&time='+str(time_interval)
# metrics['CPU_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)

# expr_pod_cpu_rabbitmq_connector = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector)
# metrics['Memory_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_memory_rabbitmq_connector)

# expr_pod_cpu_rabbitmq_connector_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas", resource="cpu",pod=~"rabbitmq.*"})&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_requested']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector_requested)

# expr_pod_rabbitmq_function = 'sum(kube_pod_container_status_ready{namespace="openfaas",pod=~"rabbitmq.*"})&time='+str(time_interval)
# metrics['pod_count_rabbitmq']=getdataprometheus(pre_url+expr_pod_rabbitmq_function)

# expr_pod_aeneas_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"aeneas.*"})&time='+str(time_interval)
# expr_pod_tocloud_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"scaling-aeneas-tocloud.*"})&time='+str(time_interval)
# expr_pod_getobject_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"getobject.*"})&time='+str(time_interval)
# print(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_aeneas']=getdataprometheus(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_tocloud']=getdataprometheus(pre_url+expr_pod_tocloud_function)
# metrics['pod_count_getobject']=getdataprometheus(pre_url+expr_pod_getobject_function)

# metrics

In [146]:
df_metrics = pd.read_csv("overall_k8s.csv")
df_metrics = df_metrics.append(metrics, ignore_index=True)

In [147]:
df_metrics.to_csv("overall_k8s.csv",index=False)
display(df_metrics)

,Average Workload TPT,Workload TPT,workload_size,workload_type,Average FET
0,11.436108,141.898997,36,36_120_k8s,11.025844
1,7.101899,133.493527,36,36_120_k8s,7.058822
2,14.603441,137.332240,45,45_120_k8s,14.350427
3,5.864947,123.858352,41,41_120_k8s,5.817637
4,11.670310,146.420508,45,45_120_k8s_0.05,11.225889


In [148]:
# Frequency
stats_df = df \
.groupby('PT') \
['PT'] \
.agg('count') \
.pipe(pd.DataFrame) \
.rename(columns = {'PT': 'frequency'})

# PDF
stats_df['pdf'] = stats_df['frequency'] / sum(stats_df['frequency'])
stats_df['cdf'] = stats_df['pdf'].cumsum()
stats_df = stats_df.reset_index()
display(stats_df)
stats_df.to_csv("stats_df_"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')
# from matplotlib import pyplot as plt

# plt.plot(stats_df['PT'],stats_df['cdf'],  linestyle='--', marker='o', color='b', label='k8s_k8s')
# #plt.plot(df['user'],df['CT'],  linestyle='--', marker='*', color='r', label='Communication Time')
# plt.grid()
# plt.legend(loc='best')
# plt.xlabel('PT (Seconds)', fontsize=18)
# plt.ylabel('CDF', fontsize=16)

,PT,frequency,pdf,cdf
0,4.0,4,0.088889,0.088889
1,5.0,3,0.066667,0.155556
2,6.0,8,0.177778,0.333333
3,7.0,6,0.133333,0.466667
4,9.0,1,0.022222,0.488889
5,10.0,1,0.022222,0.511111
6,11.0,2,0.044444,0.555556
7,12.0,4,0.088889,0.644444
8,13.0,1,0.022222,0.666667
9,14.0,3,0.066667,0.733333


In [151]:
!ssh ubuntu@172.17.141.197 "sudo kubectl get hpa -n openfaas-fn"


deployment.apps "aeneas" deleted


In [150]:
!ssh ubuntu@172.17.141.197 "sudo kubectl delete hpa/pubrabbitmq  -n openfaas-fn;sudo kubectl delete hpa/aeneas  -n openfaas-fn;sudo kubectl delete hpa/getobject  -n openfaas-fn;sudo kubectl delete hpa/scaling-aeneas-tocloud  -n openfaas-fn"

horizontalpodautoscaler.autoscaling "pubrabbitmq" deleted
horizontalpodautoscaler.autoscaling "aeneas" deleted
horizontalpodautoscaler.autoscaling "getobject" deleted
horizontalpodautoscaler.autoscaling "scaling-aeneas-tocloud" deleted
